# Data Loading, Cleansing and Algorithm Implementation

## Connecting to AWS PostgreSQL Instance

The dataset that the team leveraged is AIS data procured through the federal government. The original dataset can be obtained through the following link: https://marinecadastre.gov/ais/

In reviewing the data, the team wanted to review all of the data available. However, this dataset was incredibly large, and as the team learned all zone-data for one month represented was more than 50GB. As such, the team loaded all zones data for December 2017 in to a PostgreSQL instance on AWS to analyze and conduct our work. This dataset was a total of 64 GB. 

The team first loaded the various Python packages that were considered necessary:
* Pandas, Numpy and Math were imported to help with the data cleansing and data anlaysis. 
* SKLearn.Cluster was imported for the analytics algorithm utilized. 
* SQLAlchemy was imported due to its connectivity to cloud-hosted databases. SQLAlchemy also leverages psycopg2, which is a PostgreSQL database adapter for Python. 
* Matplotlib was imported to view some of the data as part of the analysis, to gain a better understanding of the data and what visualizations it would be capabale of. 

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import psycopg2
from sqlalchemy import create_engine
from sklearn.cluster import DBSCAN

The team leveraged the steps included in the following link to connect to a cloud-hosted PostgreSQL server: https://blog.panoply.io/connecting-jupyter-notebook-with-postgresql-for-python-data-analysis

In [ ]:
# Postgres username, password, and database name

POSTGRES_ADDRESS = 'mydbinstance.cncrogettxf7.us-east-1.rds.amazonaws.com' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'mydbinstance' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'FjhpW9aVvhM66r5Ux' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD 
POSTGRES_DBNAME = 'postgres' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
# Create the connection
team42PS = create_engine(postgres_str)

As noted in the link to panoply, once the create_engine() link has been created, we now have an object that connects to the database. We can now utilize this connection, to run an SQL query to connect to the PostgreSQL database on AWS.  

## Develop pre-processing logic on the full dataset

Given the current uncertain economic times, the team made the decision that they wanted to minimize the end-cost of this project. As such, the team wanted to perform as much pre-processing of the data as possible, so that the final cleansed datasets can be easily referenced by the visualization that will be created. However, given the size of this database (64GB) performing cleansing would require a substantial amount of compute power. Instead, the team focused on reviewing a subset of the data, manipulating it in pandas, and then converting the filters to an SQL query so that the data extracted from PostgreSQL was manageable. The subset of data selected is Zone 20. 

In [ ]:
%%time

pdPostAWS = pd.read_sql_query('''Select * from ais where \"Zone\" = 20;''', team42PS)

pdPostAWS.head()

The process of generating a dataframe based solely on Zone 20 from the PostgreSQL server took just under 11 minutes on a local SDD drive of 100GB with 16GB of RAM. The size of the dataset that we will review to determine the appropriate SQL query is 512,026 data points. 

For comparison, this script was re-run on AWS using m5.2xlarge, and the script good 3.09 CPU time, but the Wall Time stayed as 10 minutes and 40 seconds. 

In [ ]:
pdPostAWS.shape

## Review Zone 20 Data and Develop a SQL Query

The first thing to note, is that not all of the columns are needed. The only columns needed for the pre-processing task include:
* MMSI
* BaseDateTime
* LAT
* LON
* VesselType
* Status
* Length
* Width

In [ ]:
cols_to_use = ['MMSI','BaseDateTime','LAT','LON','VesselType','Status','Length','Width']

The above python code can be converted to a simple SQL query on a table. The SQL Query would be in the form of a SELECT statement, and it would be: 

SELECT 'MMSI','BaseDateTime','LAT','LON','VesselType','Status','Length','Width'. 

In [ ]:
zonde20_MOD = pdPostAWS.loc[:,cols_to_use]

print(zonde20_MOD.shape)

zonde20_MOD.head()

By reducing the table down to only the columns needed, 9 columns have been removed and the 512,026 rows were kept intact. 

Given the extensive amount of "NaN" values in this dataset, the team dropped all NaN values. Performing this work on the SQL query will be taxing to the cloud-hosted server, and so this process will be completed once the SQL query has loaded all zones in to the table. 

In [ ]:
#Drop all data with NaN

zonde20_MOD.dropna(inplace=True)

print(zonde20_MOD.shape)

zonde20_MOD.head()

Removing NaN data cuts the data almost in half, down to 313,397 rows in Zone 20. 

The team determined that there were only certain statuses that were needed to perfrom the assessment in order to create one row per trip in the dataframe. 

In [ ]:
# Filter out statuses not needed. 

bad_Status = ['at anchor', 'moored', 'power-driven vessel pushing ahead or towing alongside', 'power-driven vessel towing astern', 'under way using engine']

In [ ]:
z20_good_Status_DF = zonde20_MOD[zonde20_MOD['Status'].isin(bad_Status)].reset_index(drop=True)

print(z20_good_Status_DF.shape)

z20_good_Status_DF.head()

Removing the so-called "bad statuses" resulted in an additional 30,000 rows removed from the data. 

Additionally, the team determined that only certain vessels would be reviewed as part of this process. 

In [ ]:
#Filter out the non-cargo ships
z20_assign_vessel = z20_good_Status_DF[((z20_good_Status_DF['VesselType'] >= 70) & (z20_good_Status_DF['VesselType'] <= 89)) | (z20_good_Status_DF['VesselType'].isin({1003,1004,1016,1024}))]

z20_grouped_Set = z20_assign_vessel.groupby("MMSI")['Status'].apply(set).reset_index()

z_20_groupedMoving = z20_grouped_Set[z20_grouped_Set.apply(lambda x: len(x['Status'])>2, axis=1)]

z20_usefulShips = z_20_groupedMoving['MMSI']

z20_usefulShips.shape

In [ ]:
z20_usefulShips.head()

The above code provides us with a listing of those ships that provide us with data to leverage. It is now time to re-filter the dataset for only those ships listed. 

In [ ]:
z_20_useful_Data = z20_assign_vessel[z20_assign_vessel['MMSI'].isin(z20_usefulShips.tolist())].reset_index(drop=True)

print(z_20_useful_Data.shape)

z_20_useful_Data.head()

By reducing the data down to this list, we now have the smaller dataset that we now need to modify in order to cleanse the data for our visualization. 

## Reset the SQL Query to pull data from Zones 16 through 20

Now that we have the steps above, we are going to create a function that takes an integer (Zone) and provides out the cleansed data in the format shown above. This function should be able to run for each zone, and create the table of data cleansed. For testing purposes, we saved our first AWS extract in to an SQLite table, to perform the work on it rather than re-pulling data from AWS each time.  

In [ ]:
def usefulShips(df):
    chunkVessSet = df.groupby("MMSI")['Status'].apply(set).reset_index()

    chunkVessMoving = chunkVessSet[chunkVessSet.apply(lambda x: len(x['Status'])>2, axis=1)]

    chuckUsefulShips = chunkVessMoving['MMSI']
    
    df = df[df['MMSI'].isin(chuckUsefulShips.tolist())].reset_index(drop=True)
    
    return df

In [ ]:
import sqlite3
cnxLITE = sqlite3.connect(':memory:')
pdPostAWS.to_sql(name='zone20', con=cnxLITE)

def cleansedZone_SQLite(zoneNum):
    sqlQuery = "SELECT * FROM zone20 where Zone = " + str(zoneNum) + ";"
    
    zoneDF = pd.DataFrame(columns = cols_to_use)
    
    for chunk in pd.read_sql_query(sqlQuery , cnxLITE, chunksize=50000):
        
        chunk = chunk.loc[:,cols_to_use]
        
        chunk.dropna(inplace=True)

        chunk = chunk[chunk['Status'].isin(bad_Status)].reset_index(drop=True)
    
        chunk = chunk[((chunk['VesselType'] >= 70) & (chunk['VesselType'] <= 89)) | (chunk['VesselType'].isin({1003,1004,1016,1024}))]
        
        zoneDF = pd.concat([zoneDF,chunk], ignore_index=True)
        
    
    return zoneDF

zone20SQLite = usefulShips(cleansedZone_SQLite(20))

cnxLITE.close()

Lets now review the shape and first few rows of our SQLite database extract. 

In [ ]:
print(zone20SQLite.shape)

zone20SQLite.head()

The above code proves that our tax worked. As such, we should be able to now modify the code (to pull from AWS versus locally), and get the answer set we need. 

In [ ]:
def cleansedZone(zoneNum):
    sqlQuery = "SELECT * FROM ais where \"Zone\" = " + str(zoneNum) + ";"
    
    zoneDF = pd.DataFrame(columns = cols_to_use)
    
    for chunk in pd.read_sql_query(sqlQuery , team42PS, chunksize=50000):
        
        chunk = chunk.loc[:,cols_to_use]
        
        chunk.dropna(inplace=True)

        chunk = chunk[chunk['Status'].isin(bad_Status)].reset_index(drop=True)
    
        chunk = chunk[((chunk['VesselType'] >= 70) & (chunk['VesselType'] <= 89)) | (chunk['VesselType'].isin({1003,1004,1016,1024}))]
        
        zoneDF = pd.concat([zoneDF,chunk])
        
    return zoneDF


The next cell is where the magic happens. It is recommended that for the below cell, the user utilize a cloud based solution to run, as the datasets average about 10GB per zone (in CSV when they started). However, since the datasets are now in a PostgreSQL server and we are extracting them systematically, it may not take 10GB to download. 

Personally, when our team ran the below cell on desktop computers with 16GB of RAM, Python exceeded the 16GB of RAM in the computer and crashed. For us, this was our signal that we were starting to deal with "big data" and needed to derive a solution that leveraged some sort of framework to leverage enhanced computing power as well as additional RAM. 

The team will run the below cell on AWS for purposes of demonstrating the end-to-end functionality of the pre-processing workflow. Once the data is cleansed, the team will extract the new dataframe to a CSV file and store it (so that someone can review the end data without the need of an EC2 instance), but the code will still continue to work in the event soemone wants to run on EC2. 

The team leveraged the guidance on the following medium article regarding how to implement Jupyter on an EC2 system: https://chrisalbon.com/aws/basics/run_project_jupyter_on_amazon_ec2/

Based on EC2 specs, and the limitations imposed on AWS Starter Education accounts, the team will implement this code on an m5.2xlarge instance on EC2, which costs (on average) about 38,4 cents per hour. The expectation will be that with the transfer costs 1 cent per GB, the overall costs for pulling the data and extracting it for the Visualizations will cost under \$5.00. 

Note that the below cells should NOT be run locally, and should only be run on cloud instances. Our Team has run the below cell to demonstrate our usage of cloud computing strategies, but we also exported the final dataframe to csv, so that it can be loaded locally. 

### AWS EC2 m5.2xlarge Implementation

In [ ]:
%%time

listOfZones = [16,17,18,19,20]

allZoneData = [cleansedZone(x) for x in listOfZones]

In [ ]:
allZoneData = usefulShips(pd.concat([x for x in allZoneData]))

The above cell took 1 hour 26 minutes and 41 seconds on a m5.2xlarge machine. As such, the team saved this file down for the benefit of the end user. 

In [ ]:
allZoneData.to_csv('CleanFinalDatasetZones20-16.csv')

If the user is running the script locally, the team recommends utilizing the csv file that was provided in the GitHub repository. This csv file will load just the data necessary. Further in order to leverage multiple processors and speed the process up, the team will load the csv in to a dask dataframe.

In [2]:
df = pd.read_csv('CleanFinalDatasetZones20-16.csv')

If the file was run through AWS, please link to "df" below. 

In [ ]:
df = allZoneData

The size the dataframe is:

In [3]:
df = df.iloc[:,1:]

In [4]:
print(df.shape)

(5614782, 8)


In [5]:
df.head()

,MMSI,BaseDateTime,LAT,LON,VesselType,Status,Length,Width
0,367341010,2017-12-01 00:03:24,46.14598,-84.03378,1004.0,under way using engine,192.03,20.81
1,311017900,2017-12-01 00:01:03,43.48887,-87.53384,1004.0,under way using engine,190.01,23.60
2,636014410,2017-12-01 00:01:13,28.54872,-88.46215,1004.0,under way using engine,293.20,40.00
3,367082230,2017-12-01 00:00:23,45.96762,-85.87182,1004.0,moored,221.90,23.79
4,367127000,2017-12-01 00:01:00,28.61452,-89.63410,1024.0,under way using engine,187.43,27.46


The cleansed dataset results in 5,614,782 unique nodes to leverage. The important piece to note here is that this is just a node graph, and now we need to convert that to a tibble. 

## Sort and Group Ships based on MMSI

In order to create the specific tibbles, we first need to sort the data so that we can group and create the table correctly. 

In [6]:
df = df.sort_values(['MMSI','BaseDateTime'],ascending=True)

Next step is to determine when a ship changes status. We have two columns we like "MMSI" and "Status". We are looking for when the next row's Status does not equal the current row, so we propose adding one column to our usefulData_Sorted frame that pulls in the next row's status.

However, we need to be careful about the following edge case: https://drive.google.com/file/d/168ZbsBsB793YJMRVbc-fGuinRjlItI6u/view?usp=sharing

In [7]:
df['match_MMSI'] = ~(df.MMSI.eq(df.MMSI.shift()))
df['match_Status'] = ~(df.Status.eq(df.Status.shift()))

df = df[(df['match_MMSI']) | (df['match_Status'])]

df.shape

(6995, 10)

In [8]:
df.head()

,MMSI,BaseDateTime,LAT,LON,VesselType,Status,Length,Width,match_MMSI,match_Status
4015429,209008000,2017-12-11 07:23:13,45.59057,-73.50271,1004.0,under way using engine,184.93,23.7,True,True
2829094,209008000,2017-12-12 18:52:24,43.22754,-79.21757,1004.0,moored,184.93,23.7,False,True
2935600,209008000,2017-12-14 05:29:15,43.22787,-79.21765,1004.0,under way using engine,184.93,23.7,False,True
2948403,209008000,2017-12-14 09:04:51,43.29329,-79.82692,1004.0,at anchor,184.93,23.7,False,True
3218833,209008000,2017-12-18 02:32:51,43.29588,-79.83053,1004.0,under way using engine,184.93,23.7,False,True


Looking at the above dataframe, we have two issues of note:

1. Ships that begin with a status of "under way using engine" are one edge we need to consider. This means that the ship is "en route"...to somewhere that we don't care about. Thus, we need to ensure that the last row for any MMSI in our table is either "moored" or "at anchor".
2. The corrollary to the first bullet point is a status where it is "under way using engine". In order to start a trip, we need to be either "moored" or "at anchor". 

In [9]:
notMoving_Status = ['moored','at anchor']

df = df[df['Status'].isin(notMoving_Status)]

def CargoTanker(vessel):
    if (vessel >= 70 and vessel <= 79) or (vessel in {1003,1004,1016}):
        return 'Cargo'
    else:
        return 'Tanker'
    
temp_df = df.copy()
temp_df['VesselType'] = temp_df.apply(lambda row: CargoTanker(row['VesselType']), axis=1)

badCols = ['match_MMSI','match_Status']

finalData = temp_df.drop(columns=badCols)

finalData.head(5)

,MMSI,BaseDateTime,LAT,LON,VesselType,Status,Length,Width
2829094,209008000,2017-12-12 18:52:24,43.22754,-79.21757,Cargo,moored,184.93,23.7
2948403,209008000,2017-12-14 09:04:51,43.29329,-79.82692,Cargo,at anchor,184.93,23.7
3222890,209008000,2017-12-18 03:43:49,43.27275,-79.78643,Cargo,moored,184.93,23.7
4779592,209008000,2017-12-24 02:49:25,44.94164,-75.04328,Cargo,at anchor,184.93,23.7
4803852,209008000,2017-12-24 13:00:42,44.98705,-74.78600,Cargo,moored,184.93,23.7


In [10]:
print(finalData.shape)

(3324, 8)


In [11]:
df = finalData

By taking the above steps, we have now gotten our test dataset down from 5,00,000+ rows to a manageable 6,995 rows with good data.

## TEU Implementation Aglorithm

Now that the team has a cleansed dataset, the data needs to be converted in to one tibble per trip. Right now the data is tall and skinny (each row is point of the graph). In order to develop a tibble (short and fat) of the data, each row needs to have the necessary data for one single trip. 

The model that is being uitilzed is a hub-and-spoke model. Thus, one single trip will consist of a departure from a node, travel to a departure hub, travel to an arrival hub and then arrival to the final spoke. However, right now the code is only implemented at the node level and so the hubs need to be developed.

In order to effectively calculate the hubs, the team was determined to use DBSCAN in sklearn.cluster. To do this, a numpy array would be calculated, that would take LON and LAT from the dataframe for use in the algorithm. 

In [12]:
#Create Ports based on proximity of ships anchoring/mooring near eachother
#Create a port_reference that maps a port to a Lat,Lon

#Original Code
'''
lon = df['LON'].to_list()
lat = df['LAT'].to_list()
Coords = np.array([[i,j] for i,j in zip(lon,lat)])'''

Coords = np.array(df.loc[:,['LON','LAT']])

In order to ensure the code worked as efficeintly as possible in sklearn, the team created the Coordinate as a numpy array in order to leverage some of the efficiencies of numpy. 

The main portion of the algorithm that the team wanted to optimize, was to determine what the appropriate epsilon factor in DBSCAN that would result in the lowest carbon output. As such, the code in order to calculate the CO2 emmissions needed to be embedded within functions that could run iteratively over all of the data through various epsilon data points. The next two code cells summarize some of those functions that were generalized. The first cell includes helper functions for the main calculation cell (the detail behind this calculation can be reviewed in the Team's final project paper). The second code block is the actual function code that takeas the node trips dataframe (pulled from AWS above), and determine the hubs to travel to. 

In [13]:
#Given a ships length and width, calculate the TEU of the ship
def CalcTEU(length, width):
    area = width * length
    return (0.317688908*area) + (0.0000206756025*(area**2)) - 219.2003311

#Given the segments, return the total amount of TEU for all ships taking that segment
def mapHubTEU(segment, SEG_temp_total):
    total = SEG_temp_total[SEG_temp_total['Segment'] == segment]['Individual_TEU'].to_list()[0]
    return total

#Given the starting lat and lon of 2 points, calculate the "as a crow flies" distance
def distCalc(start_LAT, start_LON, end_LAT, end_LON): 
    lat1 = start_LAT
    lat2 = end_LAT
    lon1 = start_LON
    lon2 = end_LON
    
    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return 6371 * c

#Given the TEU of the individual ship and the TEU of the combined cargo (if applicable), return the total CO2 of the trip
def carbonCalc(ton_TEU, CO2_TEU,start_LAT, start_LON, end_LAT, end_LON):
    if CO2_TEU < 18000:
        CO2 = (-0.00432978571*CO2_TEU) + (0.000000139479467*(CO2_TEU**2)) + 36.15242554 #gCO2/tonne-km
    else:
        CO2 = 3.1
    dist = distCalc(start_LAT, start_LON, end_LAT, end_LON)
    tonage = ton_TEU * 15
    return CO2 * tonage * dist


In [14]:
def clusterPorts(orig_DF, epsil_Cluster):
    clustering = DBSCAN(eps=epsil_Cluster, min_samples=1).fit(Coords)

    orig_DF['Port_ID'] = clustering.labels_
    port_reference = orig_DF.groupby('Port_ID').mean()[['LON','LAT']]

    port_reference.columns = ['CENTER_LON','CENTER_LAT']

    finalData_withPorts = pd.merge(orig_DF, port_reference, how='left', left_on='Port_ID',right_index=True)

    uniqueTrips = finalData_withPorts.copy()
    
    uniqueTrips.columns = ['MMSI_E',"BaseDateTime_E","LAT_E","LON_E","VesselType","Status_E","Length", "Width","HUBPORT_E","HUBPORT_LON_E", "HUBPORT_LAT_E"]

    columnOrder = ["VesselType","Length", "Width",'MMSI_E',"BaseDateTime_E","LAT_E","LON_E","Status_E","HUBPORT_E","HUBPORT_LON_E", "HUBPORT_LAT_E"]

    uniqueTrips = uniqueTrips[columnOrder]

    uniqueTrips['MMSI_S'] = uniqueTrips.MMSI_E.shift()
    uniqueTrips['BaseDateTime_S'] = uniqueTrips.BaseDateTime_E.shift()
    uniqueTrips['LAT_S'] = uniqueTrips.LAT_E.shift()
    uniqueTrips['LON_S'] = uniqueTrips.LON_E.shift()
    uniqueTrips['Status_S'] = uniqueTrips.Status_E.shift()
    uniqueTrips['HUBPORT_S'] = uniqueTrips.HUBPORT_E.shift()
    uniqueTrips['HUBPORT_LON_S'] = uniqueTrips.HUBPORT_LON_E.shift()
    uniqueTrips['HUBPORT_LAT_S'] = uniqueTrips.HUBPORT_LAT_E.shift()

    uniqueTrips.dropna(inplace=True)

    uniqueTrips['Valid'] = uniqueTrips['MMSI_S']==uniqueTrips['MMSI_E']

    final_Unique_Trips = uniqueTrips[uniqueTrips['Valid']]

    final_Unique_Trips.columns= ['VesselType', 'Length', 'Width', 'MMSI', 'BaseDateTime_TripEnd', 'LAT_SPOKEEndPort','LON_SPOKEEndPort', 'Status_End', 'ENDHUBPORT_PortID', 'ENDHUBPORT_LON', 'ENDHUBPORT_LAT','MMSI_Start', 'BaseDateTime_Start', 'LAT_SPOKEStartPort', 'LON_SPOKEStartPort', 'Status_Start',
           'StartHUBPORT_PortID', 'StartHUBPORT_LON', 'StartHUBPORT_LAT', 'Valid']

    finalMappedData = final_Unique_Trips.drop(columns=['MMSI_Start','Valid','Status_End','Status_Start'])

    orderFinal = ['MMSI', 'VesselType', 'Length', 'Width','BaseDateTime_Start',
           'LAT_SPOKEStartPort', 'LON_SPOKEStartPort', 'StartHUBPORT_PortID',
           'StartHUBPORT_LON', 'StartHUBPORT_LAT' , 'BaseDateTime_TripEnd',
           'LAT_SPOKEEndPort', 'LON_SPOKEEndPort', 'ENDHUBPORT_PortID',
           'ENDHUBPORT_LON', 'ENDHUBPORT_LAT']

    finalMappedData = finalMappedData[orderFinal]

    finalMappedData['Segment'] = finalMappedData.apply(lambda row: (int(row['StartHUBPORT_PortID']),int(row['ENDHUBPORT_PortID'])), axis=1)
    finalMappedData['Individual_TEU'] = finalMappedData.apply(lambda row: CalcTEU(row['Length'],row['Width']), axis=1)
    
    seg_TEU_total = finalMappedData.groupby('Segment').agg({'Individual_TEU':sum}).reset_index()
    finalMappedData['Hub_TEU'] = finalMappedData.apply(lambda row: mapHubTEU(row['Segment'],seg_TEU_total), axis=1)
    
    finalMappedData['CO2_SpokeStart'] = finalMappedData.apply(lambda row: carbonCalc(row['Individual_TEU'], row['Individual_TEU'], row['LAT_SPOKEStartPort'], row['LON_SPOKEStartPort'], row['StartHUBPORT_LAT'], row['StartHUBPORT_LON']), axis=1)
    finalMappedData['CO2_SpokeEnd'] = finalMappedData.apply(lambda row: carbonCalc(row['Individual_TEU'], row['Individual_TEU'], row['LAT_SPOKEEndPort'], row['LON_SPOKEEndPort'], row['ENDHUBPORT_LAT'], row['ENDHUBPORT_LON']), axis=1)
    finalMappedData['CO2_Hub_Hub'] = finalMappedData.apply(lambda row: carbonCalc(row['Individual_TEU'], row['Hub_TEU'], row['StartHUBPORT_LAT'], row['StartHUBPORT_LON'], row['ENDHUBPORT_LAT'], row['ENDHUBPORT_LON']), axis=1)

    return finalMappedData


Now that the bulk of the equations have been built to perform the necessary calculations, we should test running the equations on one epsilon number to see that the results are as expected. We will test on epsilon of 0.1  

In [15]:
%%time

epsilon_Clustering_test = 0.1
testData = clusterPorts(df, epsilon_Clustering_test)
carbon_total = sum([sum(testData['CO2_SpokeStart'].to_list()), sum(testData['CO2_SpokeEnd'].to_list()), sum(testData['CO2_Hub_Hub'].to_list())])
carbon_total

CPU times: user 2.24 s, sys: 3.82 ms, total: 2.25 s
Wall time: 2.25 s


395077734887.82996

In [16]:
testData.head()

,MMSI,VesselType,Length,Width,BaseDateTime_Start,LAT_SPOKEStartPort,LON_SPOKEStartPort,StartHUBPORT_PortID,StartHUBPORT_LON,StartHUBPORT_LAT,...,LON_SPOKEEndPort,ENDHUBPORT_PortID,ENDHUBPORT_LON,ENDHUBPORT_LAT,Segment,Individual_TEU,Hub_TEU,CO2_SpokeStart,CO2_SpokeEnd,CO2_Hub_Hub
2948403,209008000,Cargo,184.93,23.7,2017-12-12 18:52:24,43.22754,-79.21757,0.0,-79.267991,43.235996,...,-79.82692,1,-79.806127,43.284009,"(0, 1)",1570.343393,4854.103055,2.932176e+06,1.381011e+06,1.905052e+07
3222890,209008000,Cargo,184.93,23.7,2017-12-14 09:04:51,43.29329,-79.82692,1.0,-79.806127,43.284009,...,-79.78643,1,-79.806127,43.284009,"(1, 1)",1570.343393,37071.447419,1.381011e+06,1.418112e+06,0.000000e+00
4779592,209008000,Cargo,184.93,23.7,2017-12-18 03:43:49,43.27275,-79.78643,1.0,-79.806127,43.284009,...,-75.04328,2,-75.037810,44.942870,"(1, 2)",1570.343393,3320.302314,1.418112e+06,3.159901e+05,2.322491e+08
4803852,209008000,Cargo,184.93,23.7,2017-12-24 02:49:25,44.94164,-75.04328,2.0,-75.037810,44.942870,...,-74.78600,3,-74.779583,44.987990,"(2, 3)",1570.343393,3320.302314,3.159901e+05,3.604924e+05,1.149182e+07
4819003,209008000,Cargo,184.93,23.7,2017-12-24 13:00:42,44.98705,-74.78600,3.0,-74.779583,44.987990,...,-73.91337,4,-73.957450,45.299436,"(3, 4)",1570.343393,1570.343393,3.604924e+05,3.303609e+06,5.119809e+07


To run the clustering algorithm for one iteration at an epsilon of 0.1, it took 3 seconds (on average), and the output appears to be reasonable. As such, we can now leverage these code blocks to process through the various epsion rates to determine the most effective rate. 

In [17]:
epsilon_Clustering = [0.0000000001, 0.00001, 0.0001, 0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1,1.25]

The below cell takes a long time to run, and we ran it to pull down the numbers in to this Jupyter notebook. Note that if you run the next cell, you should expect that it could take two minutes. 

In [18]:
%%time

finalMappedData = [clusterPorts(df, x) for x in epsilon_Clustering]

CPU times: user 1min 8s, sys: 34.7 ms, total: 1min 8s
Wall time: 1min 8s


In [19]:
def calcCarbonTotal(temp_DF):
    return sum([sum(temp_DF['CO2_SpokeStart'].to_list()), sum(temp_DF['CO2_SpokeEnd'].to_list()), sum(temp_DF['CO2_Hub_Hub'].to_list())])

In [21]:
arrayOfCarbon = [calcCarbonTotal(x) for x in finalMappedData]

In order to see the various carbon outputs along with their epsilon values, we need to create one final dataframe taht joins the epsilon values and the carbon values. 

In [22]:
dataToConvert = {'Epsiolon':epsilon_Clustering,'Total_Carbon':arrayOfCarbon}
df_CarbonOutput = pd.DataFrame(dataToConvert)

In [23]:
df_CarbonOutput

,Epsiolon,Total_Carbon
0,1.000000e-10,6.084259e+11
1,1.000000e-05,6.084259e+11
2,1.000000e-04,6.060637e+11
3,1.000000e-03,5.747221e+11
4,5.000000e-03,5.147208e+11
5,1.000000e-02,4.909102e+11
6,2.000000e-02,4.588488e+11
7,3.000000e-02,4.418731e+11
8,4.000000e-02,4.235782e+11
9,5.000000e-02,4.084754e+11


We have now completed the pre-processing of the data for our algorithm. The code below will extract the various carbon-trip tabels at the various epsion values. In order to get the data in to our DASH tables, the team extracted each of the carbon outputs to individual csv files for consumption. 

In [ ]:
j = 0

for i in epsilon_Clustering:
    fileName = 'clusteredDF_' + str(i) + '.csv'
    finalMappedData[j].to_csv(fileName)
    j += 1